In [15]:
import requests 
import json
import pandas as pd
from datetime import datetime

In [16]:
# Replace with your actual API key
api_key = "XLC9cIQDZOgmCiMCT43D1umf6VeOFuGU"

# Base URL for the API
base_url = "https://legislation.nysenate.gov/api/3/bills/"

# Mapping columns to city tracker
column_mappings = {
    "Last Updated": None,  # date tracker is exported
    "Session": "session",  # session that bill was introduced
    "Chamber": "billType_desc",  # chamber that bill was introduced in
    "Bill Number": "basePrintNo",  # bill number
    "Date Introed": "publishedDateTime",  # this date will be reformatted
    "Automated Status": "status_statusDesc",  # status of the bill
    "Sponsor": "sponsor_member_fullName", # full name of the sponsoring member
    "Co Sponsor Count": "amendments_items_coSponsors",  # result.amendments.items..coSponsors.size
    "Co Sponsor Count Change": None,  #need to add some math logic for this column
    "Bill Last Changed": "status_actionDate", #last change to the bill
    "Summary": "summary", #short summary of bill
    "Committee": "status_committeeName",  # committee name
    "Co Sponsors": None,  # name of co-sponsors
    "Notes": None,  
    "Laws Impacted": None,  # result.amendments.items..relatedLaws.items.AMEND.items
    "Bill Link": None,  # add the link to the bill URL
    "Priority Level": None  # copying from city tracker
}

In [17]:
def fetch_bills_by_year(api_key, session_year, committee_filter=None, limit=100):
    """
    Notes: Functions gets all the bills and then filters them out based on specified committee names

    """
   
    # gather all the bills in the 2025 session

    offset = 0  # The first page offset is typically 0 or 1, depending on the API
    all_bills = []

    while True:
        url = f"{base_url}{session_year}?key={api_key}&limit={limit}&offset={offset}"

        try:
            response = requests.get(url)

            if response.status_code == 200:
                bill_data = response.json()

                if "result" in bill_data and "items" in bill_data["result"]:
                    bills = bill_data["result"]["items"]
                    print(f"Fetched {len(bills)} bills with offset {offset}.")

                    if not bills:
                        print(f"No more bills found. Stopping.")
                        break

                    all_bills.extend(bills)
                    offset += limit
                else:
                    print(f"No bills found for session year {session_year}.")
                    break
            else:
                print(f"Failed to retrieve bills. Status code: {response.status_code}")
                print(f"Response: {response.text}")
                break
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            break

    # Apply committee filter
    if committee_filter:
        all_bills = [bill for bill in all_bills if 
                     bill.get("status", {}).get("committeeName") and
                     bill.get("status", {}).get("committeeName") in committee_filter]

    print(f"Total bills fetched: {len(all_bills)}")
    return all_bills

In [18]:
def flatten_json(nested_json, parent_key='', sep='_'):
    """
    Flatten a nested JSON into a flat dictionary.
    """
    items = []
    for k, v in nested_json.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_json(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                items.extend(flatten_json(item, f"{new_key}_{i}", sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [19]:
def get_column_data(item, column_mappings):
    """
    Create a row of data based on the column mappings.
    """
    row_data = {}

    for column, key in column_mappings.items():
        if key:
            # Check if the key exists in the flattened JSON data and add it to the row
            value = item.get(key, "")

            # If the column is 'Date Introed', format it as 'YYYY-MM-DD'
            if column == "Date Introed" and value:
                try:
                    # Convert to date format (YYYY-MM-DD)
                    formatted_date = datetime.fromisoformat(value).date()
                    row_data[column] = formatted_date
                except ValueError:
                    row_data[column] = value  # If it can't be converted, leave the original value
            else:
                row_data[column] = value
        else:
            # calc the last updated column
            if column == "Last Updated":
                row_data[column] = datetime.now().strftime("%Y-%m-%d")  # Set today's date for 'Last Updated'
            else:
                row_data[column] = ""

    return row_data

In [ ]:

def export_to_excel(bill_data, filename="NY_Senate_bills_data"):
    """
    Export the bill data to an Excel file with two tabs: Senate and Assembly.
    The final filename will include a timestamp.
    """
    flattened_data = []
    for item in bill_data:
        flattened_item = flatten_json(item)
        row_data = get_column_data(flattened_item, column_mappings)
        flattened_data.append(row_data)
    
    df = pd.DataFrame(flattened_data)
    
    # Sort the DataFrame by 'Bill Last Changed' if the column exists
    if "Bill Last Changed" in df.columns:
        df["Bill Last Changed"] = pd.to_datetime(df["Bill Last Changed"], errors='coerce').dt.date
        df = df.sort_values(by="Bill Last Changed", ascending=False)
    
    # Split data into Senate and Assembly DataFrames if available
    if "Chamber" in df.columns:
        senate_data = df[df['Chamber'] == 'Senate']
        assembly_data = df[df['Chamber'] == 'Assembly']
    else:
        senate_data = pd.DataFrame()
        assembly_data = pd.DataFrame()
    
    if not df.empty:
        # Append timestamp and .xlsx extension to create a unique filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        full_filename = f"{filename}_{timestamp}.xlsx"
        with pd.ExcelWriter(full_filename) as writer:
            if not senate_data.empty:
                senate_data.to_excel(writer, sheet_name="Senate", index=False)
            if not assembly_data.empty:
                assembly_data.to_excel(writer, sheet_name="Assembly", index=False)
        print(f"Data exported to {full_filename}")
    else:
        print("No data to export to Excel.")


In [21]:
def main():
    session_year = 2025  #specify session year
    
    # Filter for multiple committees
    committee_filter = ["Elections", "Election Law"]

    # get the bills
    bill_data = fetch_bills_by_year(api_key, session_year, committee_filter=committee_filter)

    if bill_data:
        # Export the bill data to Excel
        export_to_excel(bill_data)
    else:
        print("No data retrieved from the API.")

if __name__ == "__main__":
    main()

Fetched 100 bills with offset 0.
Fetched 100 bills with offset 100.
Fetched 100 bills with offset 200.
Fetched 100 bills with offset 300.
Fetched 100 bills with offset 400.
Fetched 100 bills with offset 500.
Fetched 100 bills with offset 600.
Fetched 100 bills with offset 700.
Fetched 100 bills with offset 800.
Fetched 100 bills with offset 900.
Fetched 100 bills with offset 1000.
Fetched 100 bills with offset 1100.
Fetched 100 bills with offset 1200.
Fetched 100 bills with offset 1300.
Fetched 100 bills with offset 1400.
Fetched 100 bills with offset 1500.
Fetched 100 bills with offset 1600.
Fetched 100 bills with offset 1700.
Fetched 100 bills with offset 1800.
Fetched 100 bills with offset 1900.
Fetched 100 bills with offset 2000.
Fetched 100 bills with offset 2100.
Fetched 100 bills with offset 2200.
Fetched 100 bills with offset 2300.
Fetched 100 bills with offset 2400.
Fetched 100 bills with offset 2500.
Fetched 100 bills with offset 2600.
Fetched 100 bills with offset 2700.
Fetc